In [ ]:
! pip install -q diffusers

In [ ]:
# import torch
# from diffusers import FluxPipeline

In [ ]:
# pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16)#.to("cuda")
# pipe.enable_model_cpu_offload() #save some VRAM by offloading the model to CPU. Remove this if you have enough GPU power

In [ ]:
# pipe.save_pretrained('flux_schnell')

In [ ]:
# prompt = "space photograph of a blue bird falling throught the atmosphere, with the rising sun in the horizon, in the background.jpg"
# image = pipe(
#     prompt,
#     guidance_scale=0.0,
#     num_inference_steps=5,
#     max_sequence_length=256,
#     generator=torch.Generator("cpu").manual_seed(0)
# ).images[0]

# image

In [ ]:
# image.save("flux-schnell.png")

## using Stable diffusion cascade

In [ ]:
import torch
from diffusers import (
    StableCascadeDecoderPipeline,
    StableCascadePriorPipeline,
    StableCascadeUNet,
)

prompt = "an image of a white snowflake falling through space"
negative_prompt = ""

prior_unet = StableCascadeUNet.from_pretrained("stabilityai/stable-cascade-prior", subfolder="prior_lite")
decoder_unet = StableCascadeUNet.from_pretrained("stabilityai/stable-cascade", subfolder="decoder_lite")

prior_unet.save_pretrained('stable_cascade_prior_unet')
decoder_unet.save_pretrained('stable_cascade_decoder_unet')

In [ ]:
prior = StableCascadePriorPipeline.from_pretrained("stabilityai/stable-cascade-prior", prior=prior_unet)
decoder = StableCascadeDecoderPipeline.from_pretrained("stabilityai/stable-cascade", decoder=decoder_unet)

decoder.save_pretrained('stable_cascade_decoder')
prior.save_pretrained('stable_cascade_prior')

prior.enable_model_cpu_offload()
prior_output = prior(
    prompt=prompt,
    height=1024,
    width=1024,
    negative_prompt=negative_prompt,
    guidance_scale=4.0,
    num_images_per_prompt=1,
    num_inference_steps=20
)

prior_output

In [ ]:
decoder.enable_model_cpu_offload()
decoder_output = decoder(
    image_embeddings=prior_output.image_embeddings,
    prompt=prompt,
    negative_prompt=negative_prompt,
    guidance_scale=0.0,
    output_type="pil",
    num_inference_steps=10
).images[0]

decoder_output

In [ ]:
decoder_output.save("cascade.png")